In [1]:
%matplotlib qt5
from process_files import *
from func_science import *

qt.qpa.xcb: X server does not support XInput 2


In [8]:
#
config = configobj.ConfigObj('config.ini')
line =config['line']
scdir = config['science']['directory']
imdir = config['science'][line+'/imdir']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line, 0))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wave_range = np.float64(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.float64(settings[linestr]['NWavePoints'])-1)
#
rotang1 = np.float64(config['targetplate']['rotang1'])
rotang2 = np.float64(config['targetplate']['rotang2'])
#
mag = np.float64(config['targetplate'][line+'/mag'])
shift1_y = np.float64(config['targetplate'][line+'/shift1_y'])
shift1_x = np.float64(config['targetplate'][line+'/shift1_x'])
shift2_y = np.float64(config['targetplate'][line+'/shift2_y'])
shift2_x = np.float64(config['targetplate'][line+'/shift2_x'])
pol1_shifts = [shift1_y, shift1_x, 0, 0]
pol2_shifts = [shift2_y, shift2_x, 0, 0]

In [6]:
sc_files = sorted(os.listdir(scdir+os.sep+line))[0:ncyc]
# flats
ff_pol1 = np.float64(pf.open(config['flats'][line+'/pol1'])[0].data)
ff_pol2 = np.float64(pf.open(config['flats'][line+'/pol2'])[0].data)
ff_pol = np.float64(pf.open(config['flats'][line+'/pol'])[0].data)
ff_pol1 = ff_pol[:,:,0:4]
ff_pol2 = ff_pol[:,:,4::]
# pol1 calib
pc1 = pf.open(config['pcalibration'][line+'/pol1'])
modmat1 = pc1[1].data
pc1.close()
demodmat1 = np.linalg.inv(modmat1)
# pol2 calib
pc2 = pf.open(config['pcalibration'][line+'/pol2'])
modmat2 = pc2[1].data
pc2.close()
demodmat2 = np.linalg.inv(modmat2)
#
dir_tree = scdir.split(os.sep)
l0dir = os.sep.join(dir_tree[0:-1])+os.sep+'L0'
if not (os.path.exists(l0dir)): os.mkdir(l0dir)
l0dir += os.sep+dir_tree[-1]
if not (os.path.exists(l0dir)): os.mkdir(l0dir)
l0dir += os.sep+line
if not (os.path.exists(l0dir)): os.mkdir(l0dir)
config['science'][line+'/l0dir'] = l0dir
config.write()
#
l0subdir = l0dir+os.sep+'stokes'
if not (os.path.exists(l0subdir)): os.mkdir(l0subdir)
print('Science data will be saved to ', l0subdir)

Science data will be saved to  /arch/1/VTT/hellride/raw/20230504/L0/Science_0745/Fe_I_6302/stokes


In [10]:
for i in tqdm.tqdm(range(140,141)):
    imsubdir = imdir + os.sep +f'{i:04}'
    imlist = [imsubdir+os.sep+f for f in sorted(os.listdir(imsubdir)) if '.fits' in f]
    #
    sc0 = pf.open(imlist[0])[0].data
    sc0 = np.pad(sc0, ((6,6),(7,7)), mode='constant')
    sc1, sc2 = [], []
    for m in range(nmod*nwav):
        sc1_ = pf.open(imlist[m+1])[0].data
        sc1.append(np.pad(sc1_,((6,6),(7,7)),mode='constant'))
        sc2_ = pf.open(imlist[m+1+nmod*nwav])[0].data
        sc2.append(np.pad(sc2_,((6,6),(7,7)),mode='constant'))
    sc1, sc2 = np.array(sc1), np.array(sc2)
    #
    Y, X = sc0.shape
    sc1 = np.reshape(np.moveaxis(sc1,0,2), [Y,X,nmod,nwav], order='F')
    sc2 = np.reshape(np.moveaxis(sc2,0,2), [Y,X,nmod,nwav], order='F')
    sc0[sc0==0] = sc0.mean()
    sc1[sc1==0] = sc1.mean()
    sc2[sc2==0] = sc2.mean()
    # 
    sc1_demod = np.einsum('ijkl,mk->ijml', sc1, demodmat1)
    # sc1_demod[:,:,1::] /= sc1_demod[:,:,0:1] 
    sc2_demod = np.einsum('ijkl,mk->ijml', sc2, demodmat2)
    # sc2_demod[:,:,1::] /= sc2_demod[:,:,0:1] 
    # #
    sc_name = l0subdir + os.sep + f'{i:04}_'+sc_files[i].split('_')[-2]+'.fits'
    hdu1 = pf.PrimaryHDU(sc0)
    hdu2 = pf.ImageHDU(sc1_demod)
    hdu3 = pf.ImageHDU(sc2_demod)
    hdul = pf.HDUList([hdu1,hdu2,hdu3])
    hdul.writeto(sc_name, overwrite=True)

100%|██████████| 1/1 [00:26<00:00, 26.59s/it]


In [22]:
show_img_series(sc1_demod[:,:,0], fps=2)

In [1]:
sc1_demod.shape

NameError: name 'sc1_demod' is not defined

In [ ]:
imsubdir = imdir + os.sep+ 'xtalk'
xt1 = [imsubdir+os.sep+f for f in os.listdir(imsubdir) if 'cam1' in f][0]
xt1 = pf.open(xt1)[0].data
xt1 = np.moveaxis(np.moveaxis(xt1,0,3),0,3)

In [ ]:
show_img_series(xt1[:,:,0], fps=2)

In [ ]:
os.listdir(imsubdir)

In [ ]:
sc1_ = 1.0*sc1_demod
sc1_[:,:,1::] = sc1_demod[:,:,1::]/sc1_demod[:,:,0:1]
sc1_[:,:,0] /= sc1_[:,:,0].mean()
sc2_ = 1.0*sc2_demod
sc2_[:,:,1::] = sc2_demod[:,:,1::]/sc2_demod[:,:,0:1]
sc2_[:,:,0] /= sc2_[:,:,0].mean()

In [ ]:
#
line = 'Fe_I_6173'
iline = get_line_num(settings, line, 0)
config = configobj.ConfigObj('config.ini')
dkdir = config['darks']['directory']
scdir = config['science']['directory']
settings = [f for f in os.listdir(scdir) if 'settings' in f]
settings = scdir + os.sep + settings[0]
settings = configobj.ConfigObj(settings)
# Other numbers
linestr = 'Line_' + str(get_line_num(settings, line, 0))
nacc = int(settings[linestr]['Polarimeter\\NAccumulations'])
nwav = int(settings[linestr]['NWavePoints'])
filtstr = settings[linestr]['Filter']
modstr = settings[linestr]['Polarimeter\\Modulation']
nmod = int(settings[filtstr]['Polarimeter\\'+modstr+'\\NModulations'])
nfpc = nmod*nacc*nwav
nfpw = nmod*nacc
ncyc = len(os.listdir(scdir+os.sep+line))//3
wave_range = np.float64(settings[linestr]['WaveScanRange'])
wave_step = wave_range/(np.float64(settings[linestr]['NWavePoints'])-1)
# Time stamps
# Time stamps
im0ind, im1ind, im2ind = read_time_stamps_obs(scdir, iline)
im0ind_ = im0ind.reshape([ncyc, nfpc])
im1ind_ = im1ind.reshape([ncyc, nfpc])
im2ind_ = im2ind.reshape([ncyc, nfpc])
im0ind_ -= im0ind_[:,0:1]
im1ind_ -= im1ind_[:,0:1]
im2ind_ -= im2ind_[:,0:1]
im0ind = im0ind_.reshape([ncyc, nwav, nacc, nmod])%nmod
im1ind = im1ind_.reshape([ncyc, nwav, nacc, nmod])%nmod
im2ind = im2ind_.reshape([ncyc, nwav, nacc, nmod])%nmod
#
rotang1 = np.float64(config['targetplate']['rotang1'])
rotang2 = np.float64(config['targetplate']['rotang2'])
#
mag = np.float64(config['targetplate'][line+'/mag'])
shift1_y = np.float64(config['targetplate'][line+'/shift1_y'])
shift1_x = np.float64(config['targetplate'][line+'/shift1_x'])
shift2_y = np.float64(config['targetplate'][line+'/shift2_y'])
shift2_x = np.float64(config['targetplate'][line+'/shift2_x'])
pol1_shifts = [shift1_y, shift1_x, 0, 0]
pol2_shifts = [shift2_y, shift2_x, 0, 0]
#
# darks
dk0 = np.float64(pf.open(config['darks'][line+'/bbi'])[0].data)
dk1 = np.float64(pf.open(config['darks'][line+'/pol1'])[0].data)
dk2 = np.float64(pf.open(config['darks'][line+'/pol2'])[0].data)
# flats
ff_bbi = np.float64(pf.open(config['flats'][line+'/bbi'])[0].data)
ff_bbi = ff_bbi[:,:,np.newaxis]/np.mean(ff_bbi)
ff_pol = np.float64(pf.open(config['flats'][line+'/pol'])[0].data)
ff_pol1 = ff_pol[:,:,0:4]
ff_pol2 = ff_pol[:,:,4::]


In [ ]:
for i in tqdm.tqdm(range(115,116)):
    # bbi data save
    dc0 = data_cube(scdir, line, 0, i)
    sc0 = np.uint16(16*(dc0.data - dk0)/ff_bbi)
    #
    dc1 = data_cube(scdir, line, 1, i)
    Y, X, Z = dc1.data.shape
    sc1 = np.reshape(dc1.data-dk1, [Y,X,nmod,nacc,nwav], order='F')
    sc1_add = coadd_del_accumulations (sc1, im1ind[i])
    sc1_ff = sc1_add/ff_pol[:,:,0:4,:]
    sc1_demod = np.einsum('ijkl,mk->ijml', sc1_ff, demodmat1)
    sc1_demod[:,:,1::] /= sc1_demod[:,:,0:1] 
    sc1_demod = np.flipud(sc1_demod)
    #
    dc2 = data_cube(scdir, line, 2, i)
    Y, X, Z = dc2.data.shape
    sc2 = np.reshape(dc2.data-dk2, [Y,X,nmod,nacc,nwav], order='F')
    sc2_add = coadd_del_accumulations(sc2, im2ind[i])
    sc2_ff = sc2_add/ff_pol[:,:,4::,:]
    sc2_demod = np.einsum('ijkl,mk->ijml', sc2_ff, demodmat2)
    sc2_demod[:,:,1::] /= sc2_demod[:,:,0:1] 
    #

In [ ]:
# plt.plot(sc1_demod[850,400].T)
plt.figure()
plt.plot(sc1_add[800,400].T)
#
# plt.plot(ff_pol[800,400, 0:4].T)

In [ ]:
show_img_series(sc1_demod[:,:,3], fps=2)